## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [21]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ferme-Neuve,CA,46.7001,-75.4493,48.96,68,100,6.69,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,51.48,99,89,1.99,overcast clouds
2,2,Vaini,TO,-21.2000,-175.2000,77.16,69,20,9.22,few clouds
3,3,Busselton,AU,-33.6500,115.3333,56.28,87,72,3.02,broken clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,63.00,88,40,13.80,scattered clouds
5,5,Talnakh,RU,69.4865,88.3972,5.13,94,89,7.52,overcast clouds
6,6,Lakes Entrance,AU,-37.8811,147.9810,52.16,86,97,3.06,overcast clouds
7,7,Yarada,IN,17.6500,83.2667,84.52,84,20,5.75,few clouds
8,8,Atuona,PF,-9.8000,-139.0333,79.27,72,25,23.15,scattered clouds
9,9,Bredasdorp,ZA,-34.5322,20.0403,56.12,75,1,1.66,clear sky


In [22]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Vaini,TO,-21.2000,-175.2000,77.16,69,20,9.22,few clouds
7,7,Yarada,IN,17.6500,83.2667,84.52,84,20,5.75,few clouds
8,8,Atuona,PF,-9.8000,-139.0333,79.27,72,25,23.15,scattered clouds
11,11,Alofi,NU,-19.0595,-169.9187,75.54,85,100,14.00,light rain
12,12,Rikitea,PF,-23.1203,-134.9692,78.15,74,100,22.57,overcast clouds


In [26]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
7,Yarada,IN,84.52,few clouds,17.6500,83.2667,
8,Atuona,PF,79.27,scattered clouds,-9.8000,-139.0333,
11,Alofi,NU,75.54,light rain,-19.0595,-169.9187,
12,Rikitea,PF,78.15,overcast clouds,-23.1203,-134.9692,
19,Hithadhoo,MV,84.85,overcast clouds,-0.6000,73.0833,
26,Shenzhen,CN,81.21,clear sky,22.5455,114.0683,
35,Kapaa,US,78.78,moderate rain,22.0752,-159.3190,
41,Lompoc,US,76.69,clear sky,34.6391,-120.4579,
45,Razole,IN,79.38,few clouds,16.4833,81.8333,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
#hotels["results"][0]["name"]
hotel_df.head(30)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
7,Yarada,IN,84.52,few clouds,17.6500,83.2667,Yarada Jungle Beach Resorts
8,Atuona,PF,79.27,scattered clouds,-9.8000,-139.0333,Villa Enata
11,Alofi,NU,75.54,light rain,-19.0595,-169.9187,Taloa Heights
12,Rikitea,PF,78.15,overcast clouds,-23.1203,-134.9692,People ThankYou
19,Hithadhoo,MV,84.85,overcast clouds,-0.6000,73.0833,Scoop Guest House
26,Shenzhen,CN,81.21,clear sky,22.5455,114.0683,Grand Skylight
35,Kapaa,US,78.78,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
41,Lompoc,US,76.69,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
45,Razole,IN,79.38,few clouds,16.4833,81.8333,Sterling Palavelli Godavari


In [33]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df["Hotel Name"].replace('',np.nan,inplace=True)
#hotel_df.dropna(subset=["Hotel Name"], inplace=True)

In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']

In [35]:
len(clean_hotel_df)

204

In [36]:
clean_hotel_df.head(30)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
7,Yarada,IN,84.52,few clouds,17.6500,83.2667,Yarada Jungle Beach Resorts
8,Atuona,PF,79.27,scattered clouds,-9.8000,-139.0333,Villa Enata
11,Alofi,NU,75.54,light rain,-19.0595,-169.9187,Taloa Heights
12,Rikitea,PF,78.15,overcast clouds,-23.1203,-134.9692,People ThankYou
19,Hithadhoo,MV,84.85,overcast clouds,-0.6000,73.0833,Scoop Guest House
26,Shenzhen,CN,81.21,clear sky,22.5455,114.0683,Grand Skylight
35,Kapaa,US,78.78,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
41,Lompoc,US,76.69,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
45,Razole,IN,79.38,few clouds,16.4833,81.8333,Sterling Palavelli Godavari


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))